In [1]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim

In [2]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.shape

(2557, 6)

In [5]:
y = train['avgPrice']
x = train.drop(['year', 'avgPrice'], axis=1)
test = test.drop('year', axis=1)

In [6]:
x_tensor = torch.FloatTensor(np.array(x))
y_tensor = torch.FloatTensor(np.array(y)).reshape(-1, 1)
test_tensor = torch.FloatTensor(np.array(test))

In [7]:
W = torch.zeros((x.shape[1], 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
optimizer = optim.SGD([W, b], lr=0.001)

In [9]:
epochs = 100000
for epoch in range(epochs + 1):
    
    hypothesis = x_tensor.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_tensor)** 2) 

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} shape: {} {}'.format(epoch, epochs, torch.mean(hypothesis), torch.mean(cost)))

Epoch    0/100000 shape: 0.0 12013341.0
Epoch  100/100000 shape: 3035.998046875 2173470.25
Epoch  200/100000 shape: 3044.583984375 2162982.75
Epoch  300/100000 shape: 3046.517333984375 2152960.0
Epoch  400/100000 shape: 3048.36865234375 2143224.0
Epoch  500/100000 shape: 3050.193115234375 2133765.75
Epoch  600/100000 shape: 3051.991455078125 2124575.5
Epoch  700/100000 shape: 3053.764892578125 2115645.25
Epoch  800/100000 shape: 3055.513671875 2106966.25
Epoch  900/100000 shape: 3057.237548828125 2098530.75
Epoch 1000/100000 shape: 3058.93798828125 2090330.75
Epoch 1100/100000 shape: 3060.613525390625 2082358.625
Epoch 1200/100000 shape: 3062.266845703125 2074607.625
Epoch 1300/100000 shape: 3063.89599609375 2067069.75
Epoch 1400/100000 shape: 3065.50341796875 2059739.375
Epoch 1500/100000 shape: 3067.087646484375 2052609.625
Epoch 1600/100000 shape: 3068.6513671875 2045674.125
Epoch 1700/100000 shape: 3070.192626953125 2038926.75
Epoch 1800/100000 shape: 3071.712646484375 2032362.125


In [10]:
submit = pd.read_csv('submit_sample.csv')

In [11]:

submit['Expected'] = np.array((test_tensor @ W  + b).detach())

In [12]:
submit.to_csv('submit.csv', index=False)